<a href="https://colab.research.google.com/github/drFahlan/tumor-cell-detection-using-small-and-large-FoV/blob/main/notebooks/baseline_model_development.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# setup

In [1]:
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: drfahlan (drfahlan_) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


# tissue segmentation

##train

In [2]:
import os
import time
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import psutil
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torch.utils.data import Dataset, DataLoader
import wandb
from PIL import Image
import cv2


In [9]:
# ---- Hyperparameters ----
config = {
    "batch_size": 4,
    "epochs": 150,
    "dropout":0.3613,
    "learning_rate": 0.0001649,
    "image_size": (1024, 1024),
    "patch_size": (512, 512),  # New parameter for patch-based training
    "dataset_path": "/content/drive/MyDrive/research/ocelot/dataset/ocelot2023_v1.0.1",
    "random_seed": SEED
}

In [10]:
wandb.init(project="ocelot_tissue_segmentation", name="vanillaUnet_13", reinit=True)
wandb.config.update(config)

wandb: Currently logged in as: drfahlan (drfahlan_) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


KeyboardInterrupt: 

In [4]:
# ---- Set Random Seed ----
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)


In [5]:
# ---- Paths ----
dataset_path = "/content/drive/MyDrive/research/ocelot/dataset/ocelot2023_v1.0.1"

In [6]:
# ---- Updated Augmentation ----
train_transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.Rotate(limit=30, p=0.5),
    #A.RandomCrop(height=224, width=224, p=1.0),  # Crop to 224x224
    #A.Resize(256, 256),  # Resize both image and mask to 256x256
    A.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
    ToTensorV2()
])

val_transform = A.Compose([
    #A.Resize(256, 256),  # Ensures validation images and masks are 256x256
    A.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
    ToTensorV2()
])


In [7]:
class TissueDataset(Dataset):
    def __init__(self, split, transform=None, patch_size=None):
        self.image_dir = os.path.join(config["dataset_path"], f"images/{split}/tissue")
        self.annotation_dir = os.path.join(config["dataset_path"], f"annotations/{split}/tissue")
        self.filenames = [f for f in os.listdir(self.image_dir) if f.endswith(".jpg")]
        self.transform = transform
        self.patch_size = patch_size

    def __len__(self):
        return len(self.filenames)

    def __getitem__(self, idx):
        img_name = self.filenames[idx]
        img_path = os.path.join(self.image_dir, img_name)
        ann_path = os.path.join(self.annotation_dir, img_name.replace(".jpg", ".png"))

        image = np.array(Image.open(img_path).convert("RGB"))
        annotation = cv2.imread(ann_path, cv2.IMREAD_GRAYSCALE)

        # Fix labels: Change `2` → `1` and ignore `255`
        annotation[annotation == 1] = 0
        annotation[annotation == 2] = 1

        if self.patch_size is not None:
            h, w = image.shape[:2]
            ph, pw = self.patch_size
            if h < ph or w < pw:
                raise ValueError("Image is smaller than the patch size")
            top = np.random.randint(0, h - ph + 1)
            left = np.random.randint(0, w - pw + 1)
            image = image[top:top+ph, left:left+pw]
            annotation = annotation[top:top+ph, left:left+pw]

        if self.transform:
            transformed = self.transform(image=image, mask=annotation)
            image, annotation = transformed["image"], transformed["mask"]

        return image, annotation.long()


In [11]:
# ---- Load Data ----
train_dataset = TissueDataset("train", transform=train_transform, patch_size=config["patch_size"])
val_dataset = TissueDataset("val", transform=val_transform, patch_size=config["patch_size"])

train_loader = DataLoader(train_dataset, batch_size=config["batch_size"], shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=config["batch_size"], shuffle=False)


In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Define a double convolution block: two 3x3 conv layers with ReLU activations.
class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DoubleConv, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.conv(x)

# Define the downsampling (encoder) block: max pooling followed by double convolution.
class Down(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(Down, self).__init__()
        self.down = nn.Sequential(
            nn.MaxPool2d(2),
            DoubleConv(in_channels, out_channels)
        )

    def forward(self, x):
        return self.down(x)

# Define the upsampling (decoder) block: upsampling, concatenation with corresponding encoder feature map, and double convolution.
class Up(nn.Module):
    def __init__(self, in_channels, out_channels, bilinear=True):
        super(Up, self).__init__()
        # If using bilinear interpolation for upsampling
        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        else:
            # Otherwise, use transposed convolution
            self.up = nn.ConvTranspose2d(in_channels // 2, in_channels // 2, kernel_size=2, stride=2)
        self.conv = DoubleConv(in_channels, out_channels)

    def forward(self, x1, x2):
        # x1 is the feature map from the previous decoder layer
        # x2 is the corresponding feature map from the encoder (skip connection)
        x1 = self.up(x1)
        # Adjust shape if necessary (due to cropping during encoder operations)
        diffY = x2.size()[2] - x1.size()[2]
        diffX = x2.size()[3] - x1.size()[3]
        x1 = F.pad(x1, [diffX // 2, diffX - diffX // 2,
                        diffY // 2, diffY - diffY // 2])
        # Concatenate along the channel dimension
        x = torch.cat([x2, x1], dim=1)
        return self.conv(x)

# Define the output convolution that maps to the desired number of classes.
class OutConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(OutConv, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1)

    def forward(self, x):
        return self.conv(x)

# Define the full U-Net model.
class UNet(nn.Module):
    def __init__(self, n_channels, n_classes, bilinear=True, dropout=0.1):
        """
        Args:
            n_channels: Number of input channels (e.g., 1 for grayscale, 3 for RGB)
            n_classes: Number of segmentation classes
            bilinear: Whether to use bilinear interpolation or transposed convolutions for upsampling
        """
        super(UNet, self).__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.bilinear = bilinear
        self.dropout = nn.Dropout(dropout)

        # Encoder: contracting path
        self.inc = DoubleConv(n_channels, 64)
        self.down1 = Down(64, 128)
        self.down2 = Down(128, 256)
        self.down3 = Down(256, 512)

        # Factor to adjust the number of channels when using bilinear upsampling
        factor = 2 if bilinear else 1
        self.down4 = Down(512, 1024 // factor)

        # Decoder: expansive path
        self.up1 = Up(1024, 512 // factor, bilinear)
        self.up2 = Up(512, 256 // factor, bilinear)
        self.up3 = Up(256, 128 // factor, bilinear)
        self.up4 = Up(128, 64, bilinear)

        # Final output layer
        self.outc = OutConv(64, n_classes)

    def forward(self, x):
        # Contracting path
        x1 = self.inc(x)       # [B, 64, H, W]
        x2 = self.down1(x1)    # [B, 128, H/2, W/2]
        x3 = self.down2(x2)    # [B, 256, H/4, W/4]
        x4 = self.down3(x3)    # [B, 512, H/8, W/8]
        x5 = self.down4(x4)    # [B, 1024/factor, H/16, W/16]

        # Expansive path with skip connections
        x = self.up1(x5, x4)   # [B, 512/factor, H/8, W/8]
        x = self.up2(x, x3)    # [B, 256/factor, H/4, W/4]
        x = self.up3(x, x2)    # [B, 128/factor, H/2, W/2]
        x = self.up4(x, x1)    # [B, 64, H, W]

        logits = self.outc(x)
        return logits


In [14]:
# ---- Training Setup ----
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = UNet(n_channels=3, n_classes=2, bilinear=True, dropout=config["dropout"]).to(device)
criterion = nn.CrossEntropyLoss(ignore_index=255)  # Ignore label 255
optimizer = optim.RMSprop(model.parameters(), lr=config["learning_rate"])


In [ ]:
wandb.watch(model, log="all")

In [ ]:
import numpy as np
from sklearn.metrics import f1_score, precision_score, recall_score

# ---- Training Loop with Validation, Metrics, and Best Model Saving ----
def train_model():
    best_val_loss = float("inf")
    for epoch in range(config["epochs"]):
        # ----- Training Phase -----
        model.train()
        start_time = time.time()
        total_loss, correct, total = 0, 0, 0

        for images, masks in train_loader:
            images, masks = images.to(device), masks.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, masks)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()

            total_loss += loss.item()
            preds = torch.argmax(outputs, dim=1)
            correct += (preds == masks).sum().item()
            total += masks.numel()

        train_acc = correct / total
        avg_train_loss = total_loss / len(train_loader)
        train_time = time.time() - start_time
        memory_usage = psutil.virtual_memory().percent

        # ----- Validation Phase -----
        model.eval()
        val_loss = 0
        all_preds = []
        all_labels = []
        with torch.no_grad():
            for images, masks in val_loader:
                images, masks = images.to(device), masks.to(device)
                outputs = model(images)
                loss = criterion(outputs, masks)
                val_loss += loss.item()

                preds = torch.argmax(outputs, dim=1)
                # Flatten predictions and labels for metric computation
                all_preds.append(preds.cpu().numpy().flatten())
                all_labels.append(masks.cpu().numpy().flatten())

        avg_val_loss = val_loss / len(val_loader)
        all_preds = np.concatenate(all_preds)
        all_labels = np.concatenate(all_labels)

        # Compute per-class F1, macro F1, weighted F1, recall, and precision
        per_class_f1 = f1_score(all_labels, all_preds, average=None)
        macro_f1 = f1_score(all_labels, all_preds, average='macro')
        weighted_f1 = f1_score(all_labels, all_preds, average='weighted')
        recall = recall_score(all_labels, all_preds, average=None)
        precision = precision_score(all_labels, all_preds, average=None)

        # Log metrics to wandb
        wandb.log({
            "epoch": epoch + 1,
            "train_loss": avg_train_loss,
            "train_accuracy": train_acc,
            "val_loss": avg_val_loss,
            "per_class_f1": per_class_f1.tolist(),
            "macro_f1": macro_f1,
            "weighted_f1": weighted_f1,
            "recall": recall.tolist(),
            "precision": precision.tolist(),
            "train_time": train_time,
            "memory_usage": memory_usage
        })

        print(f"Epoch [{epoch+1}/{config['epochs']}]: "
              f"Train Loss: {avg_train_loss:.4f}, Train Acc: {train_acc:.4f}, "
              f"Val Loss: {avg_val_loss:.4f}, "
              f"Time: {train_time:.2f}s, Memory: {memory_usage}%")

        # ----- Save the Best Model -----
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            torch.save(model.state_dict(), "best_model.pth")
            artifact = wandb.Artifact("best_model", type="model")
            artifact.add_file("best_model.pth")
            wandb.log_artifact(artifact)

train_model()


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Epoch [1/150]: Train Loss: 1.1318, Train Acc: 0.6298, Val Loss: 0.6996, Time: 74.13s, Memory: 22.8%


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Epoch [2/150]: Train Loss: 0.5626, Train Acc: 0.6759, Val Loss: 0.6536, Time: 71.45s, Memory: 27.5%


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


...

Epoch [126/150]: Train Loss: 0.1992, Train Acc: 0.8850, Val Loss: 0.6814, Time: 70.84s, Memory: 25.3%


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Epoch [127/150]: Train Loss: 0.1921, Train Acc: 0.8832, Val Loss: 0.5453, Time: 71.14s, Memory: 25.3%


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Epoch [128/150]: Train Loss: 0.1982, Train Acc: 0.8853, Val Loss: 0.4837, Time: 71.02s, Memory: 25.5%


debugging

In [ ]:
for images, masks in train_loader:
    print("Unique mask values:", torch.unique(masks))


Unique mask values: tensor([0, 1, 2])
Unique mask values: tensor([0, 1, 2])
Unique mask values: tensor([0, 1, 2])
Unique mask values: tensor([0, 1, 2])


KeyboardInterrupt: 

##validation

In [15]:
import os
import numpy as np
from sklearn.metrics import f1_score, precision_score, recall_score
import torch

# Initialize a new run for inference (if needed)
run = wandb.init(
    project="ocelot_tissue_segmentation",
    job_type="validation",
    name="vanillaUnet_13"
)
# Download and load the best model artifact
artifact = run.use_artifact('drfahlan_/ocelot_tissue_segmentation/best_model:v12', type='model')
artifact_dir = artifact.download()

# Instantiate your model (make sure to pass the dropout from your config)
model = UNet(n_channels=3, n_classes=2, bilinear=True, dropout=config["dropout"]).to(device)
# Load the state dict from the downloaded file (adjust the filename if needed)
model.load_state_dict(torch.load(os.path.join(artifact_dir, "best_model.pth")))
model.eval()

# ----- Validation Phase -----
val_loss = 0
all_preds = []
all_labels = []
with torch.no_grad():
    for images, masks in val_loader:
        images, masks = images.to(device), masks.to(device)
        outputs = model(images)
        loss = criterion(outputs, masks)
        val_loss += loss.item()

        preds = torch.argmax(outputs, dim=1)
        # Flatten predictions and labels for metric computation
        all_preds.append(preds.cpu().numpy().flatten())
        all_labels.append(masks.cpu().numpy().flatten())

avg_val_loss = val_loss / len(val_loader)
all_preds = np.concatenate(all_preds)
all_labels = np.concatenate(all_labels)

# Compute metrics
per_class_f1 = f1_score(all_labels, all_preds, average=None, zero_division=0)
macro_f1 = f1_score(all_labels, all_preds, average='macro', zero_division=0)
weighted_f1 = f1_score(all_labels, all_preds, average='weighted', zero_division=0)
recall = recall_score(all_labels, all_preds, average=None, zero_division=0)
precision = precision_score(all_labels, all_preds, average=None, zero_division=0)

print("Validation Loss: {:.4f}".format(avg_val_loss))
print("Per-class F1: ", per_class_f1)
print("Macro F1: ", macro_f1)
print("Weighted F1: ", weighted_f1)
print("Recall: ", recall)
print("Precision: ", precision)

# Optionally, log these metrics to wandb
wandb.log({
    "validation_loss": avg_val_loss,
    "per_class_f1": per_class_f1.tolist(),
    "macro_f1": macro_f1,
    "weighted_f1": weighted_f1,
    "recall": recall.tolist(),
    "precision": precision.tolist()
})


wandb: Downloading large artifact best_model:v12, 51.08MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:0.3
<ipython-input-15-35d165f8a9a0>:19: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to

Validation Loss: 0.3751
Per-class F1:  [0.8299681  0.80306112 0.        ]
Macro F1:  0.5443430733971888
Weighted F1:  0.779975736414316
Recall:  [0.87657486 0.79148459 0.        ]
Precision:  [0.78806722 0.81498132 0.        ]


##sweep


In [ ]:
# ---- Define Training Function for W&B Sweep ----
def train():
    wandb.init(reinit=True)  # Initialize W&B run
    config = wandb.config  # Get hyperparameters

    # ---- Define Augmentations ----
    train_transform = A.Compose([
        A.HorizontalFlip(p=0.5),
        A.Rotate(limit=30, p=0.5),
        A.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
        ToTensorV2()
    ])

    val_transform = A.Compose([
        A.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
        ToTensorV2()
    ])

    # ---- Load Data ----
    train_dataset = TissueDataset("train", transform=train_transform, patch_size=config.patch_size)
    val_dataset = TissueDataset("val", transform=val_transform, patch_size=config.patch_size)

    train_loader = DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=config.batch_size, shuffle=False)

    # ---- Model, Optimizer, Loss ----
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = UNet(n_channels=3, n_classes=2, dropout=config.dropout).to(device)
    optimizer = getattr(optim, config.optimizer)(model.parameters(), lr=config.learning_rate)
    criterion = nn.CrossEntropyLoss(ignore_index=255)

    # ---- Training Loop ----
    for epoch in range(config.epochs):
        model.train()
        total_loss, correct, total = 0, 0, 0

        for images, masks in train_loader:
            images, masks = images.to(device), masks.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, masks)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()

            total_loss += loss.item()
            preds = torch.argmax(outputs, dim=1)
            correct += (preds == masks).sum().item()
            total += masks.numel()

        train_acc = correct / total
        avg_train_loss = total_loss / len(train_loader)
        memory_usage = psutil.virtual_memory().percent

        # ---- Validation Step (Moved Inside Epoch Loop) ----
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for images, masks in val_loader:
                images, masks = images.to(device), masks.to(device)
                outputs = model(images)
                loss = criterion(outputs, masks)
                val_loss += loss.item()

        avg_val_loss = val_loss / len(val_loader)

        # ---- Log to W&B ----
        wandb.log({
            "epoch": epoch + 1,
            "train_loss": avg_train_loss,
            "train_accuracy": train_acc,
            "val_loss": avg_val_loss,  # Now tracking validation loss
            "memory_usage": memory_usage
        })

        print(f"Epoch [{epoch+1}/{config.epochs}]: Train Loss: {avg_train_loss:.4f}, Train Acc: {train_acc:.4f}, Val Loss: {avg_val_loss:.4f}, Memory: {memory_usage}%")



#FoV integration